In [ ]:
import json
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cols = ['id', 'authors', 'doi', 'title', 'abstract']
data = []
file_name = '/content/drive/MyDrive/VT/Information Storage Project Files/arxiv-metadata-oai-snapshot.json'
# file_name = '/content/drive/MyDrive/arxiv-metadata-oai-snapshot.json'

with open(file_name, encoding='latin-1') as f:
    for line in f:
        doc = json.loads(line)
        lst = [doc['id'], doc['authors'], doc['doi'], doc['title'], doc['abstract']]
        data.append(lst)

df_data = pd.DataFrame(data=data, columns=cols)

print(df_data.shape)

df_data.head()

(2436004, 5)


,id,authors,doi,title,abstract
0,0704.0001,"C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...",10.1103/PhysRevD.76.013009,Calculation of prompt diphoton production cros...,A fully differential calculation in perturba...
1,0704.0002,Ileana Streinu and Louis Theran,None,Sparsity-certifying Graph Decompositions,"We describe a new algorithm, the $(k,\ell)$-..."
2,0704.0003,Hongjun Pan,None,The evolution of the Earth-Moon system based o...,The evolution of Earth-Moon system is descri...
3,0704.0004,David Callan,None,A determinant of Stirling cycle numbers counts...,We show that a determinant of Stirling cycle...
4,0704.0005,Wael Abu-Shammala and Alberto Torchinsky,None,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,In this paper we show how to compute the $\L...


In [ ]:
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Used to securely store your API key
from google.colab import userdata

# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

model = genai.GenerativeModel('gemini-pro')

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [ ]:
def gen_questions(article):
  query = f""" Generate a short human-like user query that should return the following academic article on being searched upon.
  <ACADEMIC ARTICLE>: {article}"""

  response = model.generate_content(query, safety_settings=[
        {
            "category": "HARM_CATEGORY_DANGEROUS",
            "threshold": "BLOCK_NONE",
        },
        {
            "category": "HARM_CATEGORY_HARASSMENT",
            "threshold": "BLOCK_NONE",
        },
        {
            "category": "HARM_CATEGORY_HATE_SPEECH",
            "threshold": "BLOCK_NONE",
        },
        {
            "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
            "threshold": "BLOCK_NONE",
        },
        {
            "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
            "threshold": "BLOCK_NONE",
        },
    ])
  return response.text

In [ ]:
print(df_data['title'][0] + '\n\n' + df_data['abstract'][0])

Calculation of prompt diphoton production cross sections at Tevatron and
  LHC energies

  A fully differential calculation in perturbative quantum chromodynamics is
presented for the production of massive photon pairs at hadron colliders. All
next-to-leading order perturbative contributions from quark-antiquark,
gluon-(anti)quark, and gluon-gluon subprocesses are included, as well as
all-orders resummation of initial-state gluon radiation valid at
next-to-next-to-leading logarithmic accuracy. The region of phase space is
specified in which the calculation is most reliable. Good agreement is
demonstrated with data from the Fermilab Tevatron, and predictions are made for
more detailed tests with CDF and DO data. Predictions are shown for
distributions of diphoton pairs produced at the energy of the Large Hadron
Collider (LHC). Distributions of the diphoton pairs from the decay of a Higgs
boson are contrasted with those produced from QCD processes at the LHC, showing
that enhanced sensit

In [ ]:
question = gen_questions(df_data['title'][737] + '\n\n' + df_data['abstract'][737])
question

'How does external driving force affect the dynamics of a glassy system undergoing steady shear deformation?'

In [ ]:
type(question)

str

In [ ]:
questions = []
for i in range(100000):
  print(i, end=' ')
  question = gen_questions(df_data['title'][i] + '\n\n' + df_data['abstract'][i])
  questions.append(question)
  print("-----------DONE")

0 -----------DONE
1 -----------DONE
2 -----------DONE
3 -----------DONE
4 -----------DONE
5 -----------DONE
6 -----------DONE
7 -----------DONE
8 -----------DONE
9 -----------DONE
10 -----------DONE
11 -----------DONE
12 -----------DONE
13 -----------DONE
14 -----------DONE
15 -----------DONE
16 -----------DONE
17 -----------DONE
18 -----------DONE
19 -----------DONE
20 -----------DONE
21 -----------DONE
22 -----------DONE
23 -----------DONE
24 -----------DONE
25 -----------DONE
26 -----------DONE
27 -----------DONE
28 -----------DONE
29 -----------DONE
30 -----------DONE
31 -----------DONE
32 -----------DONE
33 -----------DONE
34 -----------DONE
35 -----------DONE
36 -----------DONE
37 -----------DONE
38 -----------DONE
39 -----------DONE
40 -----------DONE
41 -----------DONE
42 -----------DONE
43 -----------DONE
44 -----------DONE
45 -----------DONE
46 -----------DONE
47 -----------DONE
48 -----------DONE
49 -----------DONE
50 -----------DONE
51 -----------DONE
52 -----------DONE
53 

KeyboardInterrupt: 

In [ ]:
len(questions)

4001

In [ ]:
data_4001 = df_data.iloc[0:4001]

In [ ]:
data_4001['questions'] = questions

<ipython-input-24-8ee933b90b08>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_4001['questions'] = questions


In [ ]:
data_4001.head()

,id,authors,doi,title,abstract,questions
0,0704.0001,"C. Bal\'azs, E. L. Berger, P. M. Nadolsky, C.-...",10.1103/PhysRevD.76.013009,Calculation of prompt diphoton production cros...,A fully differential calculation in perturba...,How to calculate diphoton production cross sec...
1,0704.0002,Ileana Streinu and Louis Theran,None,Sparsity-certifying Graph Decompositions,"We describe a new algorithm, the $(k,\ell)$-...",Algorithmic approaches for certifying sparsity...
2,0704.0003,Hongjun Pan,None,The evolution of the Earth-Moon system based o...,The evolution of Earth-Moon system is descri...,How does the dark matter field fluid model exp...
3,0704.0004,David Callan,None,A determinant of Stirling cycle numbers counts...,We show that a determinant of Stirling cycle...,How to count single-source automata using Stir...
4,0704.0005,Wael Abu-Shammala and Alberto Torchinsky,None,From dyadic $\Lambda_{\alpha}$ to $\Lambda_{\a...,In this paper we show how to compute the $\L...,How to compute the $\Lambda_{\alpha}$ norm $\a...


In [ ]:
data_4001.to_csv('/content/drive/MyDrive/VT/Information Storage Project Files/evaluation_data_4001.csv')